## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

In [2]:
import numpy as np
import pandas as pd
from numba import vectorize
from numba import guvectorize
from numba import jit, njit
import re

In [3]:
%%time
def sa(a, b):
  return np.sum(a * b)
a1 = np.arange(1, 13).reshape(3, 4)
print(a1)
#sa = np.vectorize(sa, signature = '(i),(i)->()')
print(sa(a1, np.ones(4)))

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
78.0
CPU times: user 994 µs, sys: 0 ns, total: 994 µs
Wall time: 4.19 ms


## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [4]:
#%%timeit
A = np.random.randint(0, 1000, 1000000)
B = np.asarray([i + 100 for i in A])
c = np.mean(B)
print(c)

599.520772


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [5]:
%%time
@njit
def gen_val(sup):
  k = []
  for i in range(2000000):
    l = np.random.choice(25, 5)
    k1 = []
    for y in l:
      k1.append(sup[y])
    k.append(''.join(k1))
  return k
sup = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'G', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'Q', 'W', 'V', 'Y', 'X', 'Z', 'J', 'U']
table = pd.DataFrame(np.random.randint(1, 100, size = [2000000, 4]))
key_val = gen_val(sup)
table['key'] = key_val
table

/usr/local/lib/python3.10/dist-packages/numba/core/ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'sup' of function 'gen_val'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<timed exec>", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


CPU times: user 6.55 s, sys: 431 ms, total: 6.98 s
Wall time: 8.26 s


,0,1,2,3,key
0,55,80,1,24,SGTNN
1,77,13,50,24,QEXYE
2,90,94,6,9,QFIQZ
3,64,6,11,86,ZNFIG
4,41,50,53,89,IFNGF
...,...,...,...,...,...
1999995,70,57,6,1,WGFNQ
1999996,88,4,10,77,VABTD
1999997,55,63,38,97,LGXGA
1999998,42,73,19,2,AKKEI


In [6]:
%%time
def correct(table):
  df_filtered = table[table['key'].apply(lambda x: bool(re.match('^[A-E]{5}$', x)))]
  return df_filtered
correct(table)

CPU times: user 1.7 s, sys: 6.06 ms, total: 1.7 s
Wall time: 1.73 s


,0,1,2,3,key
439,52,66,95,15,ECCCA
4681,30,85,49,28,BECBC
5035,82,38,31,7,BADCB
6184,79,7,33,71,CEAED
6489,3,94,84,35,DBCCD
...,...,...,...,...,...
1988288,91,46,30,76,EEBED
1990151,44,73,85,81,EADAB
1994193,56,86,5,37,CEBAB
1996165,88,88,59,93,ECDBE


## Лабораторная работа 3

In [7]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 12.0 MB/s eta 0:00:00


In [8]:
%load_ext line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [9]:
with open('recipes_sample.csv') as file:
  recipes = pd.read_csv(file)
with open('reviews_sample.csv') as file:
  reviews = pd.read_csv(file)
  reviews['date'] =  pd.to_datetime(reviews['date'])

In [10]:
%%time
def a_mean(table):
  iter = table.iterrows()
  new_table = []
  for i in range(len(table)):
    el = next(iter)
    if el[1][3].year == 2010:
      new_table.append(el[1][4])
  return sum(new_table)/len(new_table)
a_mean(reviews)

CPU times: user 956 ms, sys: 7.04 ms, total: 963 ms
Wall time: 972 ms


4.476252723311547

In [11]:
%%time
def b_mean(table):
  means = []
  iter = table.iterrows()
  for i in range(len(table)):
    el = next(iter)
    means.append(el[1][4])
  return sum(means)/len(means)
reviews1 = reviews[reviews['date'].dt.year == 2010]
b_mean(reviews1)

CPU times: user 97.8 ms, sys: 2.06 ms, total: 99.8 ms
Wall time: 101 ms


4.476252723311547

In [12]:
%%time
def c_mean(table):
  k = table['rating']
  return(k.mean(0))
c_mean(reviews1)

CPU times: user 866 µs, sys: 990 µs, total: 1.86 ms
Wall time: 9.65 ms


4.476252723311547

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

Timer unit: 1e-09 s

Total time: 10.6096 s
File: /content/mprun_demos.py
Function: a_mean at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def a_mean(table):
     2         1       5166.0   5166.0      0.0    iter = table.iterrows()
     3         1        324.0    324.0      0.0    new_table = []
     4    126696   44145823.0    348.4      0.4    for i in range(len(table)):
     5    126696 9601187553.0  75781.3     90.5      el = next(iter)
     6    114602  894291357.0   7803.5      8.4      if el[1][3].year == 2010:
     7     12094   69918915.0   5781.3      0.7        new_table.append(el[1][4])
     8         1      96801.0  96801.0      0.0    return sum(new_table)/len(new_table)

In [13]:
%%time
def b_mean_ed(table):
  means = [table.iloc[i, 4] for i in range(len(table))]
  return sum(means)/len(means)
reviews1 = reviews[reviews['date'].dt.year == 2010]
b_mean_ed(reviews1)

CPU times: user 79 ms, sys: 0 ns, total: 79 ms
Wall time: 131 ms


4.476252723311547

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [14]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [15]:
%%time
get_word_reviews_count(reviews)

CPU times: user 3.66 s, sys: 31.1 ms, total: 3.69 s
Wall time: 3.72 s


{'Last': 15,
 'week': 138,
 'whole': 986,
 'sides': 51,
 'of': 19894,
 'frozen': 506,
 'salmon': 136,
 'fillet': 22,
 'was': 16322,
 'on': 6286,
 'sale': 31,
 'in': 11428,
 'my': 7972,
 'local': 117,
 'supermarket,': 3,
 'so': 8278,
 'I': 52443,
 'bought': 243,
 'tons': 29,
 '(okay,': 2,
 'only': 2577,
 '3,': 7,
 'but': 7909,
 'total': 75,
 'weight': 21,
 'over': 1739,
 '10': 426,
 'pounds).': 1,
 '': 39383,
 'This': 7304,
 'recipe': 7615,
 'is': 10073,
 'perfect': 819,
 'for': 22311,
 'fillet,': 3,
 'even': 1446,
 'though': 427,
 'it': 20185,
 'calls': 96,
 'steaks.': 19,
 'cut': 1235,
 'up': 2494,
 'the': 49056,
 'into': 1275,
 'individual': 63,
 'portions': 33,
 'and': 40287,
 'followed': 945,
 'instructions': 142,
 'exactly.': 115,
 "I'm": 1349,
 'one': 2849,
 'those': 438,
 'food': 432,
 'combining': 6,
 'diets,': 3,
 'left': 862,
 'out': 4347,
 'white': 618,
 'wine': 243,
 'added': 4024,
 'just': 4540,
 'a': 30594,
 'dash': 110,
 'vinegar': 234,
 'instead': 1894,
 '(just': 43,
 '

Уязвимые места (по профайлеру):
- {method 'append' of 'list' objects} (много элементов)
- {method 'split' of 'str' objects} (много отзывов - много создания списков)
-  Для каждого слова проверяется наличие в словаре word_reviews и, если слова там нет, создается пустой список. Это может быть затратно при большом количестве уникальных слов, так как создается много пустых списков
-  В цикле по отзывам дважды проходится по каждому слову: первый раз для создания списка word_reviews, второй раз для подсчета количества отзывов с этим словом.


In [16]:
from collections import defaultdict
def get_word_reviews_count1(df):
    word_reviews = defaultdict(list)
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = [word for word in review.split(' ')]
        for word in words:
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for word, reviews in word_reviews.items():
        word_reviews_count[word] = len(reviews)
    return word_reviews_count

In [17]:
%%time
get_word_reviews_count1(reviews)

CPU times: user 1.73 s, sys: 9.11 ms, total: 1.74 s
Wall time: 1.74 s


{'Last': 15,
 'week': 138,
 'whole': 986,
 'sides': 51,
 'of': 19894,
 'frozen': 506,
 'salmon': 136,
 'fillet': 22,
 'was': 16322,
 'on': 6286,
 'sale': 31,
 'in': 11428,
 'my': 7972,
 'local': 117,
 'supermarket,': 3,
 'so': 8278,
 'I': 52443,
 'bought': 243,
 'tons': 29,
 '(okay,': 2,
 'only': 2577,
 '3,': 7,
 'but': 7909,
 'total': 75,
 'weight': 21,
 'over': 1739,
 '10': 426,
 'pounds).': 1,
 '': 39383,
 'This': 7304,
 'recipe': 7615,
 'is': 10073,
 'perfect': 819,
 'for': 22311,
 'fillet,': 3,
 'even': 1446,
 'though': 427,
 'it': 20185,
 'calls': 96,
 'steaks.': 19,
 'cut': 1235,
 'up': 2494,
 'the': 49056,
 'into': 1275,
 'individual': 63,
 'portions': 33,
 'and': 40287,
 'followed': 945,
 'instructions': 142,
 'exactly.': 115,
 "I'm": 1349,
 'one': 2849,
 'those': 438,
 'food': 432,
 'combining': 6,
 'diets,': 3,
 'left': 862,
 'out': 4347,
 'white': 618,
 'wine': 243,
 'added': 4024,
 'just': 4540,
 'a': 30594,
 'dash': 110,
 'vinegar': 234,
 'instead': 1894,
 '(just': 43,
 '

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [23]:
## 1 Без использования массивов numpy и numba ##
%%time
def calculate_mape_v1(df):
    total_mape = 0
    count = 0
    
    for _, row in df.iterrows():
        rating = row['rating']
        if rating != 0:
            average_rating = df[df['recipe_id'] == row['recipe_id']]['rating'].mean()
            mape = abs(rating - average_rating) / average_rating
            total_mape += mape
            count += 1
    
    if count > 0:
        return total_mape / count
    else:
        return None

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.54 µs


In [24]:
## 2 Без использования массивов numpy, но с использованием numba ##

%%time
from numba import njit

@njit
def calculate_mape_v2(df):
    total_mape = 0
    count = 0
    
    for _, row in df.iterrows():
        rating = row['rating']
        if rating != 0:
            average_rating = df[df['recipe_id'] == row['recipe_id']]['rating'].mean()
            mape = abs(rating - average_rating) / average_rating
            total_mape += mape
            count += 1
    
    if count > 0:
        return total_mape / count
    else:
        return None

CPU times: user 494 µs, sys: 0 ns, total: 494 µs
Wall time: 501 µs


In [25]:
## 3 c использованием массивов numpy, но без использования numba ##
%%time
import numpy as np

def calculate_mape_v3(df):
    ratings = df['rating'].values
    recipe_ids = df['recipe_id'].values
    
    mask = (ratings != 0)
    non_zero_ratings = ratings[mask]
    non_zero_recipe_ids = recipe_ids[mask]
    
    average_ratings = np.zeros_like(non_zero_ratings)
    unique_recipe_ids = np.unique(non_zero_recipe_ids)
    
    for recipe_id in unique_recipe_ids:
        mask = (non_zero_recipe_ids == recipe_id)
        average_ratings[mask] = np.mean(non_zero_ratings[mask])
    
    mape = np.abs(non_zero_ratings - average_ratings) / average_ratings
    
    return np.mean(mape)

CPU times: user 0 ns, sys: 11 µs, total: 11 µs
Wall time: 14.3 µs


In [26]:
## 4 c использованием массивов numpy и numba ##
%%time
from numba import njit

@njit
def calculate_mape_v4(df):
    ratings = df['rating'].values
    recipe_ids = df['recipe_id'].values
    
    mask = (ratings != 0)
    non_zero_ratings = ratings[mask]
    non_zero_recipe_ids = recipe_ids[mask]
    
    average_ratings = np.zeros_like(non_zero_ratings)
    unique_recipe_ids = np.unique(non_zero_recipe_ids)
    
    for recipe_id in unique_recipe_ids:
        mask = (non_zero_recipe_ids == recipe_id)
        average_ratings[mask] = np.mean(non_zero_ratings[mask])
    
    mape = np.abs(non_zero_ratings - average_ratings) / average_ratings
    
    return np.mean(mape)

CPU times: user 368 µs, sys: 0 ns, total: 368 µs
Wall time: 377 µs


#### [версия 2]
* Уточнены формулировки задач 1, 3, 4